In [1]:
#Attribution: https://github.com/indeedlabs/indeed-python
from indeed import IndeedClient
import re
client = IndeedClient(publisher ="9124561496937265")
import time
import sqlite3
import glob
from bs4 import BeautifulSoup # For HTML parsing
import urllib # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
%matplotlib inline

In [17]:
class Indeed_JobScrap():
    
    def __init__(self, db_fname, overwrite_db, job_requirements_ls):
        self.overwrite_db = overwrite_db
        self.db_fname = db_fname
        self.job_requirements_ls = job_requirements_ls
        self.job_items = ["jobtitle", "date", "company", "country", "state", 
                          "longitude", "latitude"]
        self.tracker = [] #tracker how many consecutives job_response ar null
        self.thres = 3 #if 3 consecutives job detail request empty, stop 
        self.listing = {}
        self.table_name = "jobs"
        self.dbname = "job_listings"
        self.url = 'https://www.indeed.com/viewjob?jk='
        self.other_tables = ['degrees', 'progskills', 'qualifications', 'expertise']
        
    
    def connect_db(self):
        sql_files = glob.glob('*.db')
        if (self.db_fname in sql_files) or (overwrite_db == False):
            self.conn = sqlite3.connect(self.db_fname)
            self.c = self.conn.cursor()
        else:
            #open connection to local file
            self.conn = sqlite3.connect(self.db_fname)
            self.c = self.conn.cursor()
            # Create table
            try:
                self.c.execute("CREATE TABLE {} (jobkey, jobtitle, employer, date, country, state, longitude, latitude)".format(self.table_name))
                print('Created Table: {} in database {}'.format(self.table_name, self.dbname) )
                self.conn.commit()
            except:
                print('Unable to create table {}'.format(self.table_name))
            for this_table in self.other_tables:
                try:
                    col_name = this_table + '_req'
                    self.c.execute("CREATE TABLE {} (jobkey, {})".format(this_table, col_name))
                    print('Created Table: {} in database {}'.format(this_table, self.dbname) )
                    # Save (commit) the changes
                    self.conn.commit()
                except:
                        print('Unable to create table {}'.format(this_table))
                
                
                
                
    def save2db(self, job_entry):
        '''
        Save job_entry data to database: 
        jobkey, jobtitle, employer, data, country, state, longitude, latitude,
        degrees, qualifications, expertise, progskills
        '''
        #check if job in db
        sql_select = "SELECT {}.jobkey FROM {} WHERE {}.jobkey = '{}';".format(self.table_name, self.table_name, self.table_name, job_entry['jobkey'])
        flag_jobindb = False
        flag_error = False
        try:
            self.c.execute(sql_select)
            result = self.c.fetchall()
            if len(result) != 0:
                flag_jobindb = True
        except:
            # Insert a row of data
            print('Unable to execute SQL query: SELECT')
            flag_error = True
        print('job entry already recorded: {}'.format(flag_jobindb))        
        if flag_jobindb == False:
            #add job entry to jobs table
            sql_insert = "INSERT INTO {} VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');".format(self.table_name, 
            job_entry['jobkey'], job_entry['jobtitle'], job_entry['company'], job_entry['date'], 
            job_entry['country'], job_entry['state'], job_entry['longitude'], job_entry['latitude'])
            try:
                self.c.execute(sql_insert)
                self.conn.commit()
                print('Job entry saved in DB')
            except:
                print('Unable to save job entry in {}'.format(self.table_name))
            
            #add entry to other_tables
            for this_table in self.other_tables:
                if len(job_entry[this_table]) != 0:
                    for item in job_entry[this_table]:
                        sql_insert = "INSERT INTO {} VALUES ('{}','{}');".format(this_table, job_entry['jobkey'], item)
                        try:
                            self.c.execute(sql_insert)
                            self.conn.commit()
                        except:
                            print('Unable to save item {} in {}'.format(item, this_table))
        else:
            #update database
            sql_update = "UPDATE {} SET {}='{}', {}='{}', {}='{}', {}='{}', {}='{}', {}='{}', {}='{}' WHERE {}.jobkey={};".format(self.table_name, 
            'job_title', job_entry['jobtitle'], 'employer', job_entry['company'], 'date', job_entry['date'], 
            'country', job_entry['country'], 'state', job_entry['state'], 'longitude', 
            job_entry['longitude'], 'latitude', job_entry['latitude'], self.table_name, job_entry['jobkey'] )
            try:
                self.c.execute(sql_update)
                self.conn.commit()
                print('Job entry saved in DB')
            except:
                print('Unable to save job entry in {}'.format(self.table_name))
            
            #add/update entry to other_tables
                            

    def sanitize_text(self, text):
        try:
            text = text.decode('unicode_escape').encode('ascii', 'ignore') 
        except:
            pass

        #Remove special characters
        special_chars = "!@#$%^&*()[]{};:,./<>?\|`~=_"
        for special_char in special_chars:
            clean_text = text.replace(special_char, " ")
        #remove newline
        clean_text = " ".join(clean_text.split())
        clean_text = re.sub("[^a-zA-Z+2]"," ", clean_text)
        return clean_text
    
    
    
    def get_jobsListing(self, keywords, start, limit):
        query = ""
        if len(keywords) > 0:
            #build query
            for item in keywords:
                query = query +"+"+item
        
            params = {
                      'q' : query,
                      # 'l' : "austin",
                      'start': start,
                      'limit': limit, #maximum nbr of results returned
                      'userip' : "1.2.3.4",
                      'useragent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2)"
                     }

            return client.search(**params)
        else:
            return []                                                                                                            


                                                                                                      
                                                                                                                        
    def snif_jobDescription(self, jobkey):
        this_url = self.url+str(jobkey)
        flag_error = False
        try:
            site = urllib.request.urlopen(this_url).read() # Connect to the job posting
            soup_obj = BeautifulSoup(site, "lxml")
        except:
            print('page not found')
            flag_error = True
        
        if flag_error == False:
            # search for all <span tag
            spans = soup_obj.find_all('span', attrs={'id':"job_summary"})
            if len(spans) == 1:
                #convert soup object to text
                text = spans[0].get_text()
                #Cleaning text
                clean_text = self.sanitize_text(text)
                text_split = clean_text.split()
                return text_split
            else:
                return ""


    def extract_jobRequirements(self, jobkey):
        #get text job description
        job_description_txt = self.snif_jobDescription(jobkey)
        #save job requirements in job_req
        job_req = {}
        #Initialize
        for i in self.job_requirements_ls:
            job_req[i] = []

        #better to scroll thru text once
        for idx, word in enumerate(job_description_txt):
            w_lower = word.lower()
            
            for key in self.job_requirements_ls:
                if key == 'expertise' or key == 'qualifications':
                    if w_lower in self.job_requirements_ls[key]:
                        if len(self.job_requirements_ls[key][w_lower])==0:
                            #the word has no child in list of job req  (like 'cs':[]) 
                            if len(job_req[key])==0:
                                job_req[key] = [w_lower]
                            elif w_lower not in job_req[key]:
                                job_req[key].append(w_lower)
                        else:
                            #check the word is not the last of the job_description (otherwise next_word does not exist)
                            if idx < len(job_description_txt)-1:
                                next_word = job_description_txt[idx+1].lower()
                                #check if expression=word+next_word is in our list of job_requirements of that key
                                if next_word in self.job_requirements_ls[key][w_lower]:
                                    expression = w_lower+" "+next_word
                                    if len(job_req[key])==0:
                                        job_req[key] = [expression]
                                    elif expression not in job_req[key]:
                                        job_req[key].append(expression)
                                
                else:
                    if w_lower in self.job_requirements_ls[key]:
                        if len(job_req[key])==0:
                            job_req[key] = [w_lower]
                        elif w_lower not in job_req[key]:
                            job_req[key].append(w_lower)
        
        return job_req

                            
    def record(self, keywords, start, limit):
        while len(self.tracker) < self.thres:
            #get more responses (pagination)
            query_response = self.get_jobsListing(keywords, start, limit)
            n_entry = 0
            print(start)
            for job_entry in query_response['results']:
                job_details = {}
                n_entry += 1
                print('********* Entry # {}'.format(n_entry))
                jobkey = job_entry['jobkey']
                job_details['jobkey'] = jobkey
                #Make a query on this jobkey
                job_entry_response = client.jobs(jobkeys = (jobkey, ))
                if len(job_entry_response['results']) > 0:
                    #reset tracker
                    self.tracker = []
                    job_entry_response = job_entry_response['results'][0]
                    ##save job specs (employer, title etc...)  ########
                    for job_item in self.job_items:
                        if job_item in job_entry_response:
                            job_details[job_item] = job_entry_response[job_item]
                        else:
                            job_details[job_item] = ""
                    
                    #extract job requirements from job description
                    job_req = self.extract_jobRequirements(jobkey)
                    job_specs = {**job_details, **job_req}
                    print(job_specs)
                    self.save2db(job_specs)       
                    time.sleep(1)
                    print('Jobkey {}'.format(jobkey))
                    print()

                else:
                    self.tracker.append(1)
            
            #check consistency n_entry and limit
            start = start + limit

                
        

In [19]:
degrees_ls = ['phd', 'ms', 'bsc', 'mba', 'master', 'masters', 'bachelors', 'bachelor', 'ba', 'bs']
qualifications_ls = {'cs':[], 'ce':[], 'ee':[], 'physics':[], 'electrical': ['engineering'], 'computer': ['science'],
                     'mechanical': ['engineering'], 'industrial': ['engineering']}
expertise_ls = {'machine': ['learning'], 'Deep': ['Learning'], 'computer': ['architecture'], 'Algorithms':[], 
                'GPU': ['computing'], 'parallel': ['programming'], 'Computer': ['Vision'], 
               'predictive': ['analytics'], 'vehicle':['design', 'development', 'regulations'],
               'autonomous':['vehicles'], 'robotics':[], 'radar':[], 'kinematics':[]}
progskills_ls = ['c++', 'c', 'c#', 'python', 'opencv', 'matlab', 'php', 'java', 'ruby', 'ros', 'tensorflow', 'keras']

job_requirements_ls = {'degrees': degrees_ls, 'qualifications': qualifications_ls, 'expertise': expertise_ls, 'progskills': progskills_ls}

#keywords = ['self-driving car']
#keywords = ['self-driving', 'car']
#keywords = ['autonomous', 'cars']
#keywords = ['autonomous', 'vehicles']
#keywords = ['autonomous', 'cars']
#keywords = ['autonomous-driving']
keywords = ['autonomous-driving']
start = 0
limit = 25
db_fname = 'indeed.db'
overwrite_db = True

jobsScrap = Indeed_JobScrap(db_fname, overwrite_db, job_requirements_ls)
#connect to database
jobsScrap.connect_db()

#
jobsScrap.record(keywords, start, limit)

0
********* Entry # 1
{'progskills': ['c', 'c++', 'python', 'tensorflow', 'matlab'], 'qualifications': [], 'country': 'US', 'latitude': 37.43956, 'date': 'Tue, 02 May 2017 02:12:16 GMT', 'expertise': ['machine learning', 'parallel programming', 'radar'], 'jobtitle': 'Autopilot Internship (Summer 2017)', 'state': 'CA', 'company': 'Tesla Motors', 'degrees': ['masters', 'bachelors'], 'jobkey': '69ca8ffcbcfa7676', 'longitude': -122.14286}
job entry already recorded: True
Unable to save job entry in jobs
Jobkey 69ca8ffcbcfa7676

********* Entry # 2
{'progskills': [], 'qualifications': [], 'country': 'US', 'latitude': 40.453297, 'date': 'Thu, 11 May 2017 10:25:32 GMT', 'expertise': ['radar'], 'jobtitle': 'Systems Engineer - Active Safety', 'state': 'IN', 'company': 'Delphi', 'degrees': [], 'jobkey': '47934328f75dce9f', 'longitude': -86.13187}
job entry already recorded: True
Unable to save job entry in jobs
Jobkey 47934328f75dce9f

********* Entry # 3
{'progskills': [], 'qualifications': ['c

TypeError: 'NoneType' object is not iterable

In [ ]:
## Data analysis - search for outliers i.e jobs that have nothign to do with SDC
# 1st approach is to list all unique companies

1. remove universities
2. remove manager position

In [22]:
def connect_db():
    sql_files = glob.glob('indeed.db')
    conn = sqlite3.connect('indeed')
    c = conn.cursor()

connect_db()


In [20]:
conn = sqlite3.connect('indeed.db')
c = conn.cursor()
#select in jobs all unique values in column = employer
sql_manager = "SELECT jobkey FROM jobs WHERE jobtitle LIKE '%manager%'"
c.execute(sql_manager)
result = c.fetchall()
for jk in result:
    jk = jk[0]
    try:
        sql_delete = "DELETE FROM jobs WHERE jobkey={};".format(kj)
        c.execute(sql_delete)
    except:
        print('could not delete {} from table: jobs'.format())
c.execute(sql_manager)
result = c.fetchall()

fa7bb652d18fce60
477920bea36b9157
2afdde817310d128
8960370f01fdbf3f
fa4ae2696ba39faa
ed84ce5bb754f374
ac23a7dfebf38996
32e29fd95f9cfd42
62f2f748a639ec61
74faf342223e851e
fae1b5a1f181a309
17e376e5570d0271
1e0dc68edb237fb0
4b38db564474f682
0e06a7b3e5c97390
7ce795bd09273cea
79c2dc641f2889c1
bc25a97bc0f5c987
b56cd9a36d309613
a5e6ea0ba1b19c81
2c37dc025d920f13
a05d70e2aa8234fd
7566d884be5bea18
1604693a8780bd83
bb74eeb3791dfbe9
8f62e3f7e4ddfb21
ee711f9e43028081
251bea61562e6b5c
a4bbec94c82d34ce
359bf61bcbd908b0
67c256a4f2cf1f85
6bda83a174ffd602
4196403362d8244b
313123962f09d437
bbb29f7dec799845
7c4ce698e3e53d17
849285fc1f7aea00
33f7b63fe4dfe5b7
97f9f9f37e5f4889
4291edc6c1b0e0f9
253376c32c98aa52
479c284935a911dc
3cba11c1493a363d
572464645c677478
91b3d06d932ce496
3d36533cfe76ad4c
0a721111b17a11bd
5715c224a90b78e1
95c883d2ee9e918a
f8cc4586c3addbc7
99e8144a5614410d
70c8b9a5b6c4895e
66a5f99c13b8c84a
4e07ef706f7b7b57
b96e5c35f9299cfe
dde3e30c5b3b8139
98f1ec2a6fb94952
d74ad99467400033
c1c52f561105ec

In [ ]:
SELECT * FROM jobs WHERE jobtitle LIKE '%manager%'